In [30]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
from polygon import RESTClient
import dotenv
import os
import pandas as pd
import warnings
import heapq

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
POLYGON_API = os.getenv("POLYGON_API")

client = RESTClient(api_key=POLYGON_API)

In [40]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [43]:
'''Well now filter by dollar volume to only opearte with the 100 most liquid stocks'''
#We will compute the mean dollar volume from the past year to filter liquid stocks

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data

Price                   close        high         low        open      volume  \
date       ticker                                                               
2024-03-08 A       146.851028  150.535470  146.831177  148.608836   1588300.0   
           AAPL    169.927628  172.883671  168.146047  168.205762  76114600.0   
           ABBV    172.563904  175.738263  172.399866  173.422622   6195600.0   
           ABNB    164.910004  168.190002  163.479996  166.000000   4103600.0   
           ABT     118.544167  119.210586  118.201158  118.553969   3357100.0   
...                       ...         ...         ...         ...         ...   
2025-03-07 XYL     130.619995  130.910004  127.245003  128.020004   1282480.0   
           YUM     162.529999  163.289993  159.539993  159.550003   3410811.0   
           ZBH     107.989998  108.574997  104.349998  104.570000   1792566.0   
           ZBRA    292.690002  294.850006  285.670013  292.679993   1178179.0   
           ZTS     170.369995  170.929993  166.860001  166.860001   3291441.0   

Price                 dollarvol  
date       ticker                
2024-03-08 A         233.243488  
           AAPL    12933.973401  
           ABBV     1069.136922  
           ABNB      676.724691  
           ABT       397.964622  
...                         ...  
2025-03-07 XYL       167.517531  
           YUM       554.359108  
           ZBH       193.579199  
           ZBRA      344.841214  
           ZTS       560.762787  

[125638 rows x 6 columns]